In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.visualization
import named_arrays as na
import optika
import esis

%matplotlib notebook

astropy.visualization.quantity_support()

In [ ]:
grid = optika.vectors.ObjectVectorArray(
    wavelength=na.linspace(-1, 1, axis="wavelength", num=2) / 2,
    field=0.99 * na.Cartesian2dVectorLinearSpace(
        start=-1,
        stop=1,
        axis=na.Cartesian2dVectorArray("field_x", "field_y"),
        num=3,
    ),
    pupil=na.Cartesian2dVectorLinearSpace(
        start=-1,
        stop=1,
        axis=na.Cartesian2dVectorArray("pupil_x", "pupil_y"),
        num=15,
    ),
)

model = esis.flights.f2.optics.models.design_proposed_single(
    grid=grid,
    num_distribution=0,

)

# model = esis.flights.f1.optics.models.design_single(
#     grid=grid,
#     num_distribution=0,

# )
# model.system.transformation

In [ ]:
fig, ax = plt.subplots(
    figsize=(6, 6.5),
    constrained_layout=True
)
ax.set_aspect("equal")
model.system.plot(
    components=('z','x'),
    color="black",
    kwargs_rays=dict(
        color=na.ScalarArray(np.array(["tab:orange", "tab:blue"]), axes="wavelength"),
        label=model.system.grid_input.wavelength.astype(int),
    ),
);
handles, labels = ax.get_legend_handles_labels()
labels = dict(zip(labels, handles))
fig.legend(labels.values(), labels.keys());

In [ ]:
fig, ax = plt.subplots()

detector = model.system.surfaces_all[-1]
detector_plot = detector.plot(
    components=('x', 'y'),
    ax=ax,
    transformation=detector.transformation.inverse,
)

rays = model.system.rayfunction_default.outputs
rays = detector.transformation.inverse(rays)
na.plt.scatter(
    rays.position.x.to(u.um),
    rays.position.y.to(u.um),
    # c=na.linspace(-1, 1, 'slit', 3),
    where=rays.unvignetted,
    ax=ax,
)





In [ ]:
x = rays.position.x
x = x - x.mean(axis=('pupil_x', 'pupil_y'))
y = rays.position.y
y = y - y.mean(axis=('pupil_x', 'pupil_y'))
rms_spot_size = np.sqrt(np.square(x) + np.square(y))
rms_spot_size.mean().to(u.um)


In [ ]:
delta_z = model.field_stop.translation.z-model.grating.translation.z
delta_r = model.grating.distance_radial
theta = np.arctan2(delta_r,delta_z)
theta.to(u.deg)

In [ ]:
alpha = theta + model.grating.yaw
alpha.to(u.deg)

In [ ]:
grating_r = na.linspace(-20 * u.mm, 20 * u.mm, 11)
line_spacing = 1/model.grating.rulings.spacing(
        position=na.Cartesian3dVectorArray(
            x = na.linspace(-20 * u.mm, 20 * u.mm, 11),
            y= 0 * u.mm,
            z = 0 * u.mm,
            )
    )

fig, ax =plt.subplots()
na.plt.plot(grating_r,line_spacing.x,ax=ax)

wavelength = na.ScalarArray([465, 499] * u.AA, axes='wavelength')

beta = np.arcsin(np.sin(alpha) + wavelength.mean()/model.grating.rulings.spacing(position=na.Cartesian3dVectorArray()*u.mm).length)
beta.to(u.deg)

In [ ]:
image_z = model.detector.translation.z-model.grating.translation.z
image_r = model.detector.distance_radial - delta_r
beta = np.arctan2(image_r,image_z)-model.grating.yaw
beta.to(u.deg)

In [ ]:
object_distance = np.sqrt(
    np.square(model.grating.translation.z - model.field_stop.translation.z) + np.square(model.grating.distance_radial))
image_distance = np.sqrt(np.square(model.grating.translation.z - model.detector.translation.z) +
                         np.square(model.grating.distance_radial - model.detector.distance_radial)
                         )
magnification = image_distance / object_distance
magnification


In [ ]:
print(model.primary_mirror.translation.z)
print(model.field_stop.translation.z)
print(model.grating.translation.z)
print(model.detector.translation.z)

In [ ]:
hole_spacing = 4 * 25.4 * u.mm
hole_spacing.to(u.mm)

In [ ]:
new_primary_f = model.primary_mirror.sag.focal_length - hole_spacing * 6
new_primary_f


In [ ]:
new_fs_z = new_primary_f
new_fs_z

In [ ]:
extension_ratio = new_primary_f / model.primary_mirror.sag.focal_length
extension_ratio

In [ ]:
old_grating_z = model.grating.translation.z - model.field_stop.translation.z
print(old_grating_z)
print(model.field_stop.translation.z)
print(model.grating.translation.z)
new_grating_z = old_grating_z * extension_ratio + new_fs_z
print(new_grating_z)

In [ ]:
model.primary_mirror.sag.focal_length = new_primary_f
model.field_stop.translation.z = new_fs_z
model.grating.translation.z = new_grating_z + 30 * u.mm

In [ ]:
print(model.primary_mirror.translation.z)
print(model.field_stop.translation.z)
print(model.grating.translation.z)
print(model.detector.translation.z)

In [ ]:
model.grating.translation.z

In [ ]:
object_distance = np.sqrt(
    np.square(model.grating.translation.z - model.field_stop.translation.z) + np.square(model.grating.distance_radial))
image_distance = np.sqrt(np.square(model.grating.translation.z - model.detector.translation.z) +
                         np.square(model.grating.distance_radial - model.detector.distance_radial)
                         )
magnification = image_distance / object_distance
print(magnification)

print(object_distance)
print(image_distance)

In [ ]:
new_grating_f = image_distance * object_distance / (image_distance + object_distance)
print(-2*new_grating_f)
print(model.grating.sag.radius)

In [ ]:
print(-2*new_grating_f/model.grating.sag.radius)

In [ ]:
model.grating.sag.radius = -2 * new_grating_f.nominal * 0.9975

model.grating.yaw = -2.51 * u.deg

c0 = 1 / (1850 / u.mm)
c1 = -2.852e-5 * (u.um / u.mm)
c2 = -2.112e-7 * (u.um / u.mm ** 2)

model.grating.rulings.spacing.coefficients[0].nominal = c0
model.grating.rulings.spacing.coefficients[1].nominal = c1
model.grating.rulings.spacing.coefficients[2].nominal = c2

In [ ]:
# print(model.primary_mirror.translation.z)
# print(model.field_stop.transformation)
# print(model.grating.translation.z)
# print(model.detector.translation.z)

model.central_obscuration.translation.z = model.grating.translation.z - 25 * u.mm
model.front_aperture.translation.z = model.grating.translation.z - 100 * u.mm

In [ ]:
model.filter.radius_clear = 40 * u.mm

In [ ]:
model.detector.yaw

In [ ]:
del model.system

wavelength = na.ScalarArray([465, 499] * u.AA, axes='wavelength')
# model.grating.sag.radius = - 9.17468102e+02 * u.mm
# rays = model.system.raytrace(wavelength=wavelength)
# rays.outputs.shape

In [ ]:
fig, ax = plt.subplots(
    figsize=(6, 6.5),
    constrained_layout=True
)
# ax.set_aspect("equal")
model.system.plot(
    components=("z", "x"),
    color="black",
    kwargs_rays=dict(
        color=na.ScalarArray(np.array(["tab:orange", "tab:blue"]), axes="wavelength"),
        label=model.system.grid_input.wavelength.astype(int),
    ),
    # plot_rays_vignetted=True,
    # wavelength=wavelength,
)
handles, labels = ax.get_legend_handles_labels()
labels = dict(zip(labels, handles))
fig.legend(labels.values(), labels.keys());

In [ ]:
fig, ax = plt.subplots()

detector = model.system.surfaces_all[-1]
detector_plot = detector.plot(
    components=('x', 'y'),
    ax=ax,
    transformation=detector.transformation.inverse,
    # transformation=None,
)

rays = model.system.raytrace(wavelength=wavelength).outputs[dict(surface=-1)]

rays = detector.transformation.inverse(rays)
na.plt.scatter(
    rays.position.x,
    rays.position.y,
    where=rays.unvignetted,
)


In [ ]:
x = rays.position.x
x = x - x.mean(axis=('pupil_x', 'pupil_y'))
y = rays.position.y
y = y - y.mean(axis=('pupil_x', 'pupil_y'))
rms_spot_size = np.sqrt(np.square(x) + np.square(y))
rms_spot_size.mean().to(u.um)

In [ ]:
np.abs(rays.position[dict(wavelength=0)].x.mean()) - 8 * u.mm

In [ ]:
np.abs(rays.position[dict(wavelength=1)].x.mean()) - 8 * u.mm

In [ ]:
def esis_ii_merit(
        guesses,
        model=None,
        # wavelength = None, 
        weight=None,
        # units = None
):
    # guess = [guess*unit for guess,unit in zip(guesses,units)]
    # print(guesses)
    # grating_yaw, grating_c0, grating_c1, grating_c2, grating_roc, detector_yaw = guesses
    grating_yaw, grating_c0, grating_c1, grating_c2, grating_roc = guesses


    model.grating.yaw = grating_yaw * u.deg
    model.grating.rulings.spacing.coefficients[0] = grating_c0 * u.mm
    model.grating.rulings.spacing.coefficients[1] = grating_c1 * u.um / u.mm
    model.grating.rulings.spacing.coefficients[2] = grating_c2 * u.um / u.mm ** 2
    model.grating.sag.radius = grating_roc * u.mm
    # model.detector.yaw = detector_yaw * u.deg

    wavelength = na.ScalarArray([465, 499] * u.AA, axes='wavelength')

    del model.system
    rays = model.system.raytrace(wavelength=wavelength).outputs[dict(surface=-1)]
    detector = model.system.surfaces_all[-1]

    rays = detector.transformation.inverse(rays)

    x = rays.position.x
    x = x - x.mean(axis=('pupil_x', 'pupil_y'))
    y = rays.position.y
    y = y - y.mean(axis=('pupil_x', 'pupil_y'))

    r_squared = np.square(x) + np.square(y)
    rms_spot_size = np.sqrt(r_squared)

    t1 = np.abs(rays.position[dict(wavelength=0)].x.mean()) - 7.35 * u.mm
    t2 = np.abs(rays.position[dict(wavelength=-1)].x.mean()) - 7.35 * u.mm

    t = rms_spot_size.mean().nominal
    merit = t + weight * np.sqrt(np.square(t1.nominal) + np.square(t2.nominal))

    # print(t.value, t1.nominal.value, t2.nominal.value)
    print(merit.ndarray.value)
    return merit.ndarray.value





In [ ]:
model.grating.sag.radius

In [ ]:
from scipy.optimize import minimize, differential_evolution

guess = [
    model.grating.yaw,
    model.grating.rulings.spacing.coefficients[0].nominal,
    model.grating.rulings.spacing.coefficients[1].nominal,
    model.grating.rulings.spacing.coefficients[2].nominal,
    model.grating.sag.radius,
    # model.detector.yaw
]
guess_units = [g.unit for g in guess]
print(guess_units)

guess_values = [g.value for g in guess]
# guess_values = [-2.47016209e+00, 5.49599486e-04, -1.80866504e-05, -1.08341748e-07,
#                 -9.17468102e+02, -1.04771264e+01]
# guess_values = [-2.47016209e+00, 5.49599486e-04, -1.80866504e-05, -1.08341748e-07,
                # -9.17468102e+02]

print(guess_values)


In [ ]:
esis_ii_merit(guess_values, model=model, weight=1)

# bounds = [(-2.6, -2.45), (1 / 1900, 1 / 1800), (-3e-5, -1e-5), (-2.15e-7, -1e-7), (917, 921), (-15, -10)]
# bounds = [(-2.54, -2.48), (1 / 1860, 1 / 1840), (-2.9e-5, -2.7e-5), (-2.15e-7, -2e-7), (-917, -915)]
bounds = [(-2.6, -2.45), (1 / 1900, 1 / 1800), (-3e-5, -1e-5), (-2.15e-7, -1e-7), (-918, -914)]



# fit = minimize(esis_ii_merit, guess_values, (model, 1),
#                bounds=bounds,
#                )
# w = 15
# fit = differential_evolution(esis_ii_merit,x0=guess_values,args = (model,1),bounds=bounds,
#                              workers=w,
#                              polish=False,
#                             )

In [ ]:
# print(fit.x)

In [ ]:
guess = [-2.45125246e+00,  5.53764773e-04, -1.83548407e-05, -1.17579027e-07,
 -9.17485462e+02]

In [ ]:
# guess = [-2.53962948e+00, 5.34862656e-04, -2.10590822e-05, -1.94958850e-07,
#          9.18952281e+02, -1.22332163e+01]

# guess = [-2.47016209e+00, 5.49599486e-04, -1.80866504e-05, -1.08341748e-07,
#          9.17468102e+02, -1.04771264e+01]

# guess = [-2.50429148e+00,  5.41200713e-04 ,-2.77220822e-05 ,-2.02847671e-07,
#  -9.16287793e+02]
# # guess = [-2.53962948e+00,  0, 0, 0,
# #   9.1895221e+02, 0]

# grating_yaw, grating_c0, grating_c1, grating_c2, grating_roc = fit.x
grating_yaw, grating_c0, grating_c1, grating_c2, grating_roc = guess

model.grating.yaw = grating_yaw * u.deg
model.grating.rulings.spacing.coefficients[0] = grating_c0 * u.mm
model.grating.rulings.spacing.coefficients[1] = grating_c1 * u.um / u.mm
model.grating.rulings.spacing.coefficients[2] = grating_c2 * u.um / u.mm ** 2
model.grating.sag.radius = grating_roc * u.mm


In [ ]:
model.grating.sag

In [ ]:
del model.system
model.system.raytrace(wavelength=wavelength)

In [ ]:
fig, ax = plt.subplots(
    figsize=(6, 6.5),
    constrained_layout=True
)
ax.set_aspect("equal")
model.system.plot(
    components=("z", "x"),
    color="black",
    kwargs_rays=dict(
        color=na.ScalarArray(np.array(["tab:orange", "tab:blue"]), axes="wavelength"),
        label=model.system.grid_input.wavelength.astype(int),
    ),
    plot_rays_vignetted=False,
    # wavelength=wavelength,
)
handles, labels = ax.get_legend_handles_labels()
labels = dict(zip(labels, handles))
fig.legend(labels.values(), labels.keys());

In [ ]:
model.detector

In [ ]:
fig, ax = plt.subplots()

pixel_size = model.detector.width_pixels

detector = model.system.surfaces_all[-1]
detector_plot = detector.plot(
    components=('x', 'y'),
    ax=ax,
    transformation=detector.transformation.inverse,
    # transformation=None,
)
del model.system
print(wavelength)
rays = model.system.raytrace(wavelength=wavelength).outputs[dict(surface=-1)]

rays = detector.transformation.inverse(rays)
na.plt.scatter(
    rays.position.x,
    rays.position.y,
    where=rays.unvignetted,
)

In [ ]:
x = rays.position.x
x = x - x.mean(axis=('pupil_x', 'pupil_y'))
y = rays.position.y
y = y - y.mean(axis=('pupil_x', 'pupil_y'))
rms_spot_size = np.sqrt(np.square(x) + np.square(y))
rms_spot_size.mean(axis=('pupil_x','pupil_y')).to(u.um)

In [ ]:
np.abs(rays.position[dict(wavelength=0)].x.mean()) - 8 * u.mm

In [ ]:
np.abs(rays.position[dict(wavelength=1)].x.mean()) - 8 * u.mm

In [ ]:
import astropy.constants as const
from plasmapy.formulary.speeds import thermal_speed
thermal_speed(T=.5*u.MK,particle='Ne')


In [ ]:

#Plate Scale Calc
delta_wv = wavelength[dict(wavelength=1)]-wavelength[dict(wavelength=0)]
delta_x = rays.position.x[dict(wavelength=1)].mean() - rays.position.x[dict(wavelength=0)].mean()
delta_x = delta_x / model.detector.width_pixels * u.pixel
spectral_plate_scale = delta_wv/delta_x.to(u.pixel)
(spectral_plate_scale/wavelength*const.c).to(u.km/u.s/u.pixel)


In [ ]:
object_distance = np.sqrt(
    np.square(model.grating.translation.z - model.field_stop.translation.z) + np.square(model.grating.distance_radial))
image_distance = np.sqrt(np.square(model.grating.translation.z - model.detector.translation.z) +
                         np.square(model.grating.distance_radial - model.detector.distance_radial)
                         )
magnification = image_distance / object_distance
spatial_plate_scale = model.detector.width_pixels / (magnification*model.primary_mirror.sag.focal_length) * u.rad
print(spatial_plate_scale.to(u.arcsec))

In [ ]:
model.grating.translation

In [ ]:
model.field_stop.translation

In [ ]:
model.grating.translation - model.field_stop.translation

In [ ]:
model.grating.distance_radial

In [ ]:
model.grating.yaw

In [ ]:
model.field_stop.translation

In [ ]:
delta_z = model.field_stop.translation.z-model.grating.translation.z
delta_r = model.grating.distance_radial
theta = np.arctan2(delta_r,delta_z)
theta.to(u.deg)

In [ ]:
alpha = theta + model.grating.yaw
alpha.to(u.deg)

In [ ]:
grating_r = na.linspace(-20 * u.mm, 20 * u.mm, 11)
line_spacing = 1/model.grating.rulings.spacing(
        position=na.Cartesian3dVectorArray(
            x = na.linspace(-20 * u.mm, 20 * u.mm, 11),
            y= 0 * u.mm,
            z = 0 * u.mm,
            )
    )

fig, ax =plt.subplots()
na.plt.plot(grating_r,line_spacing.x,ax=ax)

wavelength

In [ ]:
beta = np.arcsin(np.sin(alpha) + wavelength.mean()/model.grating.rulings.spacing(position=na.Cartesian3dVectorArray()*u.mm).length)
beta.to(u.deg)

In [ ]:
image_z = model.detector.translation.z-model.grating.translation.z
image_r = model.detector.distance_radial - delta_r
beta = np.arctan2(image_r,image_z)-model.grating.yaw
beta.to(u.deg)

Mirror Specification

In [ ]:
primary_slope_error = 1 * u.urad

In [ ]:
model.primary_mirror.radius_clear

In [ ]:
sub_aperture = wavelength / np.tan(primary_slope_error)
sub_aperture = sub_aperture.to(u.mm)
sub_aperture